In [288]:
import filereader
import numpy as np
import gurobipy as gp
import scipy

In [289]:
groceries_dict = filereader.read_grocery_info()

In [290]:
recipes_dict = filereader.read_recipes()

In [291]:
nutrition_needs_dict = filereader.read_nutrition()

In [292]:
# generate the matrices that we care about
# we first need to be certain of order of stuff, so let's generate these lists of keys
groceries_list = [key for key in groceries_dict.keys()]
recipes_list = [key for key in recipes_dict.keys()]
nutrients_list = [key for key in nutrition_needs_dict.keys()]

# # generate the N matrix (but this will actually be a dictionary)
# N = {}
# for recipe, ingredients in recipes_dict.items():
#     N[recipe]

N = [
    [
        sum([groceries_dict[ingredient]["nutrients"][nutrient]/groceries_dict[ingredient]["Serving Size (g)"]*recipes_dict[recipe][ingredient] for ingredient in recipes_dict[recipe]])
    for nutrient in nutrients_list]
    for recipe in recipes_list
]

# c = np.array([groceries_dict[ingredient]["Price (CAD)"] for ingredient in groceries_list])

In [293]:
np.max(N)

6770.6

In [294]:
# need to transpose the recipes dict
recipes_dict_transpose = {}

for ik in groceries_dict.keys():
    recipes_dict_transpose[ik] = {}

for rk, rv in recipes_dict.items():
    for ik, iv in rv.items():
        recipes_dict_transpose[ik][rk] = iv

# first entry is a list, with an entry for each ingredient, containing a list of the indices of all the recipes
# the second entry is a list of lists of corresponding masses
recipes_transpose_csr_ish = [[],[]]
for ik, iv in recipes_dict_transpose.items():
    recipes_transpose_csr_ish[0].append([recipes_list.index(recipe) for recipe in iv.keys()])
    recipes_transpose_csr_ish[1].append([mass for mass in iv.values()])


In [295]:
N_dict = {}
for rk, rv in recipes_dict.items():
    nutrients_dict = {}
    for nutrient in nutrition_needs_dict.keys():
        nutrients_dict[nutrient] = 0
    for gk, gv in rv.items():
        for nk, nv in groceries_dict[gk]["nutrients"].items():
            nutrients_dict[nk] += nv*gv/groceries_dict[gk]["Serving Size (g)"]
    N_dict[rk] = nutrients_dict


In [296]:
# here, we define the cost, which is time-dependent
# for this reason, we need to be clear about the number of days we are considering

# the number of days we consider (one month)
t_max = 30
# t_max = 7

# the standard deviation for the random variation in price
price_sigma = 0.2

# costs for each item. Each is a list of 30 entries, one for each day
c_dict = {}
for gk, gv in groceries_dict.items():
    c_dict[gk] = gv["Price (CAD)"]*np.random.normal(loc=1, scale=price_sigma, size=t_max)

c_list = [gv["Price (CAD)"]*np.random.normal(loc=1, scale=price_sigma, size=t_max) for gk, gv in groceries_dict.items()]

# masses
m_dict = {}
for gk, gv in groceries_dict.items():
    m_dict[gk] = gv["Mass (g)"]

m_list = [gv["Mass (g)"] for gk, gv in groceries_dict.items()]

a_k = [nv for nk, nv in nutrition_needs_dict.items()]

In [ ]:
model = gp.Model("New problem")

## seems that summation doesn't work unless we use the Gurobi addVars
# g_vars = {}
# u_vars = {}
# for gk in groceries_dict.keys():
#     u_vars[gk] = []
#     g_vars[gk] = model.addVars(t_max, vtype=gp.GRB.INTEGER, name=gk)
#     for t in range(t_max):
#         u_vars[gk].append(model.addVars(t+1, name=f"grams of {gk} used on day {t}"))


# r_vars = {}
# for rk in recipes_dict.keys():
#     r_vars[rk] = model.addVars(t_max, name=rk)

# g_vars = model.addVars(len(groceries_list), t_max, vtype=gp.GRB.INTEGER, name="g[i][t]")
g_vars = model.addVars(len(groceries_list), t_max, name="g[i][t]")
u_vars = model.addVars(len(groceries_list), t_max, t_max, name="u[i][tau][t]")
# r_vars = model.addVars(len(recipes_list), t_max, name="r[j][t]")
r_vars = model.addMVar((len(recipes_list), t_max), name="r[j][t]")

s = model.addVars(t_max, vtype=gp.GRB.BINARY, name="s")

model.setParam("LogToConsole", 1)  # Enable console logging
model.setParam("OutputFlag", 1)    # Enable all output

model.update()


Set parameter LogToConsole to value 1
Set parameter OutputFlag to value 1


In [298]:
(np.array(a_k)*np.expand_dims(np.arange(t_max), axis=1)).shape

(30, 16)

In [299]:
np.array(a_k)

array([3.0e+03, 5.6e+01, 2.3e+03, 4.7e+03, 1.3e+03, 4.1e+02, 1.1e+01,
       1.1e+01, 9.0e+02, 1.2e+00, 1.3e+00, 1.6e+01, 1.3e+00, 2.4e+00,
       7.5e+01, 1.5e+01])

In [300]:
np.max(np.array(N).transpose())

6770.6

In [301]:
np.max(m_list)

2000.0

In [302]:
for t in range(t_max):
    model.addMConstr(np.array(N).transpose(), r_vars[:,t], '>=', np.array(a_k))

model.addConstrs((g_vars[gk, t] <= 999999*s[t] for gk in range(len(groceries_list))
                  for t in range(t_max)), 
                  name="Buy only on shopping days")

for t in range(t_max):
    model.addConstrs(
        (
            # u_vars[i,tau,t] <= g_vars[i,tau]*m_list[i] - u_vars.sum(i,tau,'*') for i in range(len(groceries_list)) for tau in range(t)
            u_vars[i,tau,t] <= g_vars[i,tau]*m_list[i] - sum(u_vars[i,tau,gamma] for gamma in range(max(0,tau-1))) for i in range(len(groceries_list)) for tau in range(t+1)
        )
    )
    
# enforce that u matches what we're using
# for ik, R_i in enumerate(recipes_transpose_csr_ish):
for i, iv in enumerate(recipes_transpose_csr_ish[0]):
    masses_required = recipes_transpose_csr_ish[1][i]
    print(f"ingredient {i}/{len(recipes_transpose_csr_ish[0])}")
    for t in range(t_max):
        # model.addConstrs((
        model.addConstr((
            sum(r_vars[iv[ind], t]*masses_required[ind]  for ind in range(len(iv))) == sum(u_vars[i, tau, t] for tau in range(
                (0 if t < int(groceries_dict[groceries_list[j]]["Shelf Life (Days)"]) 
                    else t - int(groceries_dict[groceries_list[j]]["Shelf Life (Days)"])), 
                    t+1
                ))
        ))


model.update()

ingredient 0/49
ingredient 1/49
ingredient 2/49
ingredient 3/49
ingredient 4/49
ingredient 5/49
ingredient 6/49
ingredient 7/49
ingredient 8/49
ingredient 9/49
ingredient 10/49
ingredient 11/49
ingredient 12/49
ingredient 13/49
ingredient 14/49
ingredient 15/49
ingredient 16/49
ingredient 17/49
ingredient 18/49
ingredient 19/49
ingredient 20/49
ingredient 21/49
ingredient 22/49
ingredient 23/49
ingredient 24/49
ingredient 25/49
ingredient 26/49
ingredient 27/49
ingredient 28/49
ingredient 29/49
ingredient 30/49
ingredient 31/49
ingredient 32/49
ingredient 33/49
ingredient 34/49
ingredient 35/49
ingredient 36/49
ingredient 37/49
ingredient 38/49
ingredient 39/49
ingredient 40/49
ingredient 41/49
ingredient 42/49
ingredient 43/49
ingredient 44/49
ingredient 45/49
ingredient 46/49
ingredient 47/49
ingredient 48/49


In [303]:
recipes_dict_transpose

{'Beef': {'Zurich-Style Meat Saute': 600.0,
  'Hakka-Style Meatballs': 227.0,
  'Beef and Broccoli': 227.0},
 'Mushrooms': {'Zurich-Style Meat Saute': 250.0,
  'One-Pot Chicken Tetrazzini': 227.0},
 'Onion': {'Zurich-Style Meat Saute': 150.0,
  'Galinha Caipira': 150.0,
  'Grilled Mackerel with Miso Soup and Squash': 50.0,
  'Tajine Maadnous': 150.0,
  'Exotic Ginger Cumin Chicken': 75.0,
  'Hakka-Style Meatballs': 70.0,
  'One-Pot Chicken Tetrazzini': 70.0,
  'Smoked Salmon Pasta Primavera': 70.0,
  'Three Bean Salad': 70.0,
  'Fajitas': 150.0,
  'Smoked Salmon Quiche': 75.0},
 'Flour': {'Zurich-Style Meat Saute': 8.0,
  'Spiced Apple Pancakes': 250.0,
  'Grostoli': 800.0,
  'One-Pot Chicken Tetrazzini': 24.0,
  'Smoked Salmon Pasta Primavera': 24.0,
  'Spicy Kung Pao-Style Chicken': 30.0,
  'Smoked Salmon Quiche': 220.0},
 'Cream': {'Zurich-Style Meat Saute': 200.0, 'Smoked Salmon Quiche': 240.0},
 'Parsley': {'Zurich-Style Meat Saute': 5.0,
  'Tajine Maadnous': 50.0,
  'One-Pot Chic

In [304]:
g_vars

{(0, 0): <gurobi.Var g[i][t][0,0]>,
 (0, 1): <gurobi.Var g[i][t][0,1]>,
 (0, 2): <gurobi.Var g[i][t][0,2]>,
 (0, 3): <gurobi.Var g[i][t][0,3]>,
 (0, 4): <gurobi.Var g[i][t][0,4]>,
 (0, 5): <gurobi.Var g[i][t][0,5]>,
 (0, 6): <gurobi.Var g[i][t][0,6]>,
 (0, 7): <gurobi.Var g[i][t][0,7]>,
 (0, 8): <gurobi.Var g[i][t][0,8]>,
 (0, 9): <gurobi.Var g[i][t][0,9]>,
 (0, 10): <gurobi.Var g[i][t][0,10]>,
 (0, 11): <gurobi.Var g[i][t][0,11]>,
 (0, 12): <gurobi.Var g[i][t][0,12]>,
 (0, 13): <gurobi.Var g[i][t][0,13]>,
 (0, 14): <gurobi.Var g[i][t][0,14]>,
 (0, 15): <gurobi.Var g[i][t][0,15]>,
 (0, 16): <gurobi.Var g[i][t][0,16]>,
 (0, 17): <gurobi.Var g[i][t][0,17]>,
 (0, 18): <gurobi.Var g[i][t][0,18]>,
 (0, 19): <gurobi.Var g[i][t][0,19]>,
 (0, 20): <gurobi.Var g[i][t][0,20]>,
 (0, 21): <gurobi.Var g[i][t][0,21]>,
 (0, 22): <gurobi.Var g[i][t][0,22]>,
 (0, 23): <gurobi.Var g[i][t][0,23]>,
 (0, 24): <gurobi.Var g[i][t][0,24]>,
 (0, 25): <gurobi.Var g[i][t][0,25]>,
 (0, 26): <gurobi.Var g[i][t][0,

In [305]:
# {k: v for k,v in enumerate(c_list)}
len(c_list[0])

30

In [306]:
# model.setObjective(g_vars.prod({k: v for k,v in enumerate(c_list)}) + 10*s.sum('*'))
model.setObjective(sum(g_vars[i, t]*c_list[i][t] for i in range(len(groceries_list)) for t in range(t_max)) + 10*s.sum('*'))

model.update()

In [307]:
model.optimize()

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (linux64 - "Ubuntu 20.04.6 LTS")

CPU model: Intel(R) Core(TM) i7-10750H CPU @ 2.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 26205 rows, 46140 columns and 267150 nonzeros
Model fingerprint: 0x5ccff6e1
Variable types: 44640 continuous, 1500 integer (30 binary)
Coefficient statistics:
  Matrix range     [6e-02, 1e+06]
  Objective range  [1e-01, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+03]
Found heuristic solution: objective 5276.3393733
Presolve removed 16085 rows and 37291 columns
Presolve time: 0.35s
Presolved: 10120 rows, 8849 columns, 35064 nonzeros
Variable types: 7350 continuous, 1499 integer (29 binary)

Root relaxation: objective 7.418772e+01, 5974 iterations, 0.57 seconds (0.25 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent

In [260]:
np.max(model.getA())

6770.6

In [308]:
# Initialize variables to track the largest coefficient
max_coefficient = 0
max_row = None
max_col = None

# Iterate through all constraints
for i, constr in enumerate(model.getConstrs()):
    row = model.getRow(constr)  # Sparse representation of constraint row
    for j in range(row.size()):
        coeff = row.getCoeff(j)
        if abs(coeff) > abs(max_coefficient):  # Update if this coefficient is larger
            max_coefficient = coeff
            max_row = i
            max_col = row.getVar(j).index


In [ ]:
max_coefficient

-999999.0

In [263]:
max(recipes_transpose_csr_ish[1])

[2500.0, 454.0, 907.0, 450.0, 680.0, 680.0, 454.0]

In [264]:
print(model.getVarByName("r[j][t][0,0]"))

<gurobi.Var r[j][t][0,0] (value 0.0)>


In [265]:
model.getVarByName("s")

In [309]:
# for var in model.getVars():
#     print(var)
for var in model.getVars():
        if var.x != 0:
            print(var.varName, "=", var.x)

g[i][t][3,0] = 1.0
g[i][t][3,5] = 1.0
g[i][t][3,10] = 1.0
g[i][t][3,15] = 1.0
g[i][t][3,20] = 1.0
g[i][t][3,25] = 2.0
g[i][t][8,0] = 1.0
g[i][t][8,5] = 1.0
g[i][t][8,10] = 1.0
g[i][t][8,15] = 1.0
g[i][t][8,20] = 1.0
g[i][t][8,25] = 1.0
g[i][t][14,0] = 1.0
g[i][t][14,5] = 1.0
g[i][t][14,10] = 1.0
g[i][t][14,15] = 1.0
g[i][t][14,20] = 1.0
g[i][t][14,25] = 1.0
g[i][t][15,0] = 1.0
g[i][t][15,5] = 1.0
g[i][t][15,10] = 1.0
g[i][t][15,15] = 1.0
g[i][t][15,20] = 1.0
g[i][t][15,25] = 1.0
g[i][t][16,0] = 1.0
g[i][t][16,5] = 1.0
g[i][t][16,10] = 1.0
g[i][t][16,15] = 1.0
g[i][t][16,20] = 1.0
g[i][t][16,25] = 1.0
g[i][t][19,0] = 1.0
g[i][t][19,5] = 1.0
g[i][t][19,10] = 1.0
g[i][t][19,15] = 0.9999999963556344
g[i][t][19,20] = 1.0
g[i][t][19,25] = 1.0
g[i][t][22,0] = 1.0
g[i][t][22,5] = 1.0
g[i][t][22,10] = 1.0
g[i][t][22,15] = 1.0
g[i][t][22,20] = 1.0
g[i][t][22,25] = 1.0
g[i][t][48,0] = 32.0
g[i][t][48,5] = 32.0
g[i][t][48,10] = 32.0
g[i][t][48,15] = 32.0
g[i][t][48,20] = 32.0
g[i][t][48,25] = 16.0

In [286]:
np.savetxt("daily_costs.csv", np.array(c_list), delimiter=",")

In [285]:
# g[i][t][3,0] = 2.0
# g[i][t][8,0] = 1.0
# g[i][t][14,0] = 1.0
# g[i][t][15,0] = 1.0
# g[i][t][16,0] = 1.0
# g[i][t][19,0] = 1.0
# g[i][t][22,0] = 1.0
# g[i][t][48,0] = 16.0
for i in [3, 8, 14, 15, 16, 19, 22, 48]:
    print(groceries_list[i])

Flour
Butter
Milk
Salt
Sugar
Egg
Baking Powder
Banana


In [287]:
recipes_list[6]

'Grostoli'

In [ ]:
nutrients_eaten = np.array(N[17])*16 + np.array(N[6])*3.3
for ni in range(len(nutrients_list)):
    print(f"{nutrients_list[ni]}: requires {a_k[ni]}, ate {nutrients_eaten[ni]}")

Calories (kcal): requires 3000.0, ate 17239.79
Protein (g): requires 56.0, ate 419.3354
Sodium (mg): requires 2300.0, ate 6906.94
Potassium (mg): requires 4700.0, ate 15774.328000000001
Calcium (mg): requires 1300.0, ate 1908.6560000000002
Magnesium (mg): requires 410.0, ate 1557.525
Zinc (mg): requires 11.0, ate 34.929899999999996
Iron (mg): requires 11.0, ate 52.1339
Vitamin A (µg): requires 900.0, ate 2335.4639999999995
Vitamin B1 (Thiamin) (mg): requires 1.2, ate 4.01866
Vitamin B2 (Riboflavin) (mg): requires 1.3, ate 7.448379999999999
Vitamin B3 (Niacin) (mg): requires 16.0, ate 135.188
Vitamin B6 (mg): requires 1.3, ate 13.90504
Vitamin B12 (µg): requires 2.4, ate 9.042
Vitamin C (mg): requires 75.0, ate 264.47999999999996
Vitamin E (mg): requires 15.0, ate 15.00712


In [273]:
groceries_list[14]

'Milk'

In [267]:
print(model.getVarByName("g[i][t][0,0]"))

<gurobi.Var g[i][t][0,0] (value -0.0)>


In [276]:
model.write('final_model.mps')